# Aggregate gridded data to HRU using areal averaging

### Output Format:

year month day hour minute second hru1 hru2 hru3

1996 10 1 0 0 0 0.004 0.050 0.070

1996 10 2 0 0 0 0.500 0.040 0.100

In [10]:
import rasterio as rs
import geopandas as gpd
import netCDF4

In [2]:
# list regions
regions = glob.glob('./data/nhrus/clean_AEA/nhru_*_clean.shp')

In [3]:
fl = regions[0]

In [4]:
# first generate a list of hrus and their grid cell contributions
tmp = gpd.read_file(fl)
dat = pd.DataFrame()
dat['nhruID'] = tmp.hru_id_nat.unique()
reg = fl.split('_')[-2]


In [8]:
reg = fl.split('_')[-2]

In [3]:
# load the hrap index grid and grab the cell centers
fl = './data/hrap_grid_med_wgs84.tiff' # index grid with 10 cells / actual cell for partial contributions to the HRUs

with rs.open(fl) as r:
    T0 = r.affine
    p1 = Proj(r.crs)
    A = r.read(1)
    
cols,rows = np.meshgrid(np.arange(A.shape[1]),np.arange(A.shape[0]))

T1 = T0 * Affine.translation(0.5,0.5)

rc2en = lambda r, c: (c,r)*T1

lon,lat = np.vectorize(rc2en)(rows,cols)

In [4]:
def create_geom(df):
    return Point(df.lon,df.lat)

In [5]:
n,m = lon.shape

lon.shape = n*m
lat.shape = n*m
A.shape = n*m

In [6]:
len(lon)

36029477

In [7]:
df = pd.DataFrame({'lon':lon,'lat':lat,'idx':A})

del lon; del lat; del A

In [8]:
geometry = df.apply(create_geom,axis=1)

In [9]:
df = df.drop(['lon','lat'],axis=1)
crs = {'init':'epsg:4326'}

In [10]:
df = gpd.GeoDataFrame(df,crs=crs,geometry=geometry)

In [1]:
df.to_file('./data/index.geojson',driver='GeoJSON')

NameError: name 'df' is not defined

In [10]:
p2 = Proj(proj='latlong',datum='WGS84')
longs,lats = transform(p1,p2,eastings,northings)

In [14]:
n,m = longs.shape

longs.shape = (n*m)
lats.shape = (n*m)
A.shape = (n*m)

In [15]:
df = pd.DataFrame({'lon':longs,'lat':lats,'idx':A})

In [16]:
del longs
del lats
del A

In [ ]:
geometry = [Point(x,y) for x,y in zip(df.lon,df.lat)]
df = df.drop(['lon','lat'],axis=1)
crs = {'init':'epsg:4326'}